In [27]:
import requests
import os
import json
import time
from tqdm import tqdm

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = "AAAAAAAAAAAAAAAAAAAAAHLrbgEAAAAA777OwnP8h1tYMwnBUv9JW84UIuU%3DAmrUujZvDvPBHsf8yN0tsEx0D4OBK0Cj50hnAapZo44PMuimGa"


def create_url(ids):
    tweet_fields = "tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,source,text,withheld"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "ids=" + ids
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def crawl_save(tweet_ids, output_folder):
    tweet_id_list = []
    for tweet_id in tweet_ids.readlines():
        tweet_id_list.extend(tweet_id.strip().split(","))
    tweet_amount = len(tweet_id_list)
    start_id = 0
    end_id = 100
    single_response_tweet_id = []
    while start_id < tweet_amount:
        single_response_tweet_id.append(",".join(tweet_id_list[start_id:end_id]))
        start_id = end_id
        end_id = start_id + 100

    crawl_count = 0
    for ids in tqdm(single_response_tweet_id):
        url = create_url(ids)
        json_responses = connect_to_endpoint(url)
        for response in json_responses["data"]:
            json.dump(response, open(output_folder + str(response["id"]) + ".json", "w"))
        crawl_count += 1
        if crawl_count % 290 == 0:
            time.sleep(1100)
    
        
        
    

def main():
#     url = create_url()
#     json_response = connect_to_endpoint(url)
#     print(json.dumps(json_response, indent=4, sort_keys=True))
    print("Start crawling train tweet")
    crawl_save(open("data/train.data.txt", "r"), "data/train_tweets/")
    print("Finish crawling train tweet")
    
    print("Start crawling dev tweet")
    crawl_save(open("data/dev.data.txt", "r"), "data/dev_tweets/")
    print("Finish crawling dev tweet")
    
    print("Start crawling covid tweet")
    crawl_save(open("data/covid.data.txt", "r"), "data/covid_tweets/")
    print("Finish crawling covid tweet")


if __name__ == "__main__":
    main()

  0%|          | 0/305 [00:00<?, ?it/s]

Start crawling train tweet


  0%|          | 0/106 [00:00<?, ?it/s]

Finish crawling train tweet
Start crawling dev tweet


  0%|          | 0/2547 [00:00<?, ?it/s]

Finish crawling dev tweet
Start crawling covid tweet


100%|██████████| 2547/2547 [3:02:25<00:00,  4.30s/it]     

Finish crawling covid tweet
